In [1]:
import MySQLdb
import node_edge_gen as neg
import networkx as nx

from geopy.distance import great_circle
from geopy.distance import vincenty

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets.species_distributions import construct_grids
from sklearn.neighbors import KernelDensity

In [151]:
# DEFINE SQL LOG IN 
HOST = 'localhost'
USER = 'root'
PASSWD = ''
DATABASE = 'bk_map'

#CONNECT TO MySQL
db_connect = MySQLdb.connect(
    host = HOST,
    user = USER,
    passwd = PASSWD,
    db = DATABASE)

cur = db_connect.cursor()


In [75]:
cur.execute(""" SELECT longitude, latitude FROM bk_crime 
    WHERE crime IN ('ROBBERY', 'GRAND LARCENY','FELONY ASSAULT', 'MURDER') 
    AND latitude BETWEEN 40.569 AND 40.7847
    AND longitude BETWEEN -74.0500 AND -73.5899
    AND month = 8
    """)
crime_location = cur.fetchall()

zip(*crime_location)
plt.scatter(*zip(*crime_location))
plt.show()

In [76]:
cur.execute('''SELECT nodeid, latitude, longitude FROM bk_nodes
                ''')  
nodes_sql = cur.fetchall()


In [61]:
def plot_nodes(nodes):
    for d in nodes:
        plt.scatter(nodes[d][0],nodes[d][1])


In [77]:
cur.execute(""" SELECT latitude FROM bk_crime 
    WHERE crime IN ('ROBBERY', 'GRAND LARCENY','FELONY ASSAULT', 'MURDER') 
    AND latitude BETWEEN 40.569 AND 40.7847
    AND longitude BETWEEN -74.0500 AND -73.5899
    AND month = 8
    """)
crime_lat = cur.fetchall()
crime_lat_list = []
for lat in crime_lat:
    crime_lat_list.append(lat)
cur.execute(""" SELECT longitude FROM bk_crime 
    WHERE crime IN ('ROBBERY', 'GRAND LARCENY','FELONY ASSAULT', 'MURDER') 
    AND latitude BETWEEN 40.569 AND 40.7847
    AND longitude BETWEEN -74.0500 AND -73.5899
    AND month = 8
    """)
crime_lng = cur.fetchall()
crime_lng_list = []
for lng in crime_lng:
    crime_lng_list.append(lng)

cur.execute("""SELECT total FROM bk_crime 
    WHERE crime IN ('ROBBERY', 'GRAND LARCENY','FELONY ASSAULT', 'MURDER') 
    AND latitude BETWEEN 40.569 AND 40.7847
    AND longitude BETWEEN -74.0500 AND -73.589
    AND month = 8
    """)
crime_total = cur.fetchall()
crime_total_list = []
for total in crime_total:
    crime_total_list.append(total)


In [78]:
crime_lat_np = np.array(crime_lat_list)
crime_lng_np = np.array(crime_lng_list)
crime_total_np = np.array(crime_total_list)

data = np.hstack([crime_lat_np, crime_lng_np, crime_total_np])
#data[:,2]

In [79]:

Xtrain = data[:,0:2]
ytrain = data[2]


# Set up the data grid for the contour plot
xgrid = np.linspace(-74.1, -73.65, num=1000)
ygrid = np.linspace(40.5, 40.8, num=1000)
X, Y = np.meshgrid(xgrid, ygrid)

xy = np.vstack([Y.ravel(), X.ravel()]).T

# Plot map of with distributions of each species
fig = plt.figure()
    # construct a kernel density estimate of the distribution
print(" - computing KDE in spherical coordinates")
kde = KernelDensity(bandwidth=0.002,
                    kernel='gaussian')
kde.fit(Xtrain, y = ytrain)

 # evaluate only on the land: -9999 indicates ocean
Z = np.exp(kde.score_samples(xy))
Z = Z.reshape(X.shape)

    # plot contours of the density
levels = np.linspace(0, Z.max(), 25)
plt.contourf(X, Y, Z, levels=levels, cmap=plt.cm.Reds)
plt.title('BK CRIME')

#zip(*crime_location)
#plt.scatter(*zip(*crime_location))

plt.show()
#plot_nodes(nodes_sql)

#plt.savefig('201509_crime_kde.png')

 - computing KDE in spherical coordinates


In [70]:
node_dic = {}
for node in nodes_sql:
    node_dic[node[0]] = node[1:] 


In [47]:
heatmap, xedges, yedges = np.histogram2d(data[:,0], data[:,1], bins=100)
#extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap, origin='lower')#, extent=extent)
plt.show()

In [107]:
levels = np.linspace(0, Z.max(), 25)
plt.contourf(X, Y, Z, levels=levels, cmap=plt.cm.Reds)
plt.title('BK CRIME')

#for d in node_dic:
#    plt.scatter(node_dic[d][1],node_dic[d][0])
    
#zip(*crime_location)
#plt.scatter(*zip(*crime_location), color='black')


plt.show()

In [48]:
cur.close()
db_connect.close()

In [142]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx
    
def crimeheatmap_to_node(node_dic, lnggrid, latgrid, colormap):
    crime_dic = {}
    for node in node_dic:
        tmpx = find_nearest(lnggrid,float(node_dic[node][1])) #lng
        tmpy = find_nearest(latgrid,float(node_dic[node][0])) #lat
        #print tmpx, tmpy
        #crime_dic[node] = node_dic[node][0], node_dic[node][1], colormap[tmpx,tmpy]
        crime_dic[node] = colormap[tmpx,tmpy]        
    return crime_dic
    

In [143]:
crime_dic = crimeheatmap_to_node(node_dic, xgrid,ygrid,Z)


In [146]:

def distance_btw_nodes(node1,node2,node_dic):
    return great_circle(node_dic[node1], node_dic[node2]).miles

def accident_weight_btw_nodes(node1, node2, weight_dic):
##    node1 = str(node1)
##    node2 = str(node2)
    if node1 in weight_dic and node2 in weight_dic:
        return weight_dic[node1] + weight_dic[node2]
    elif node1 in weight_dic and node2 not in weight_dic:
        return weight_dic[node1]
    elif node1 not in weight_dic and node2 in weight_dic:
        return weight_dic[node2]
    else:
        return 0

def make_weights_crime(node_dict,edge_dict,feature_dict, feature_factor):
    tmp_edge = {}
    for edge in edge_dict:
        wei = accident_weight_btw_nodes(edge_dict[edge][0], edge_dict[edge][1], feature_dict)
        weight = ((feature_factor*wei) + 1)* distance_btw_nodes(edge_dict[edge][0],edge_dict[edge][1],node_dic)
        tmp_edge[edge] = edge_dict[edge][0], edge_dict[edge][1], weight
    return tmp_edge
    #    #print wei
    #return

In [137]:
#cur.execute('''SELECT edgeid, node1, node2 FROM bk_edges
#                ''')  
#edges_sql = cur.fetchall()
#edge_dic = {}
#for edge in edges_sql:
#    edge_dic[edge[0]]= edge[1:]
    


In [147]:
crime_weights  =  make_weights_crime(node_dic,edge_dic,crime_dic, 1)



In [148]:
crime_weights

{'5679816': ('42476036', '42476712', 0.048636083733516466),
 '56764174': ('42475331', '42475336', 0.079004866337939308),
 '5676294': ('42473937', '42473945', 0.088980288914160002),
 '56764172': ('42475325', '42475328', 0.59669376402875007),
 '56764173': ('42475328', '42475331', 0.68366465986107217),
 '56764170': ('42475320', '42475322', 1.0447970134523201),
 '56764171': ('42475322', '42475325', 0.9180619122880922),
 '239626770': ('42497720', '42497721', 0.061750412796128681),
 '322783828': ('3295739694', '2484497672', 0.047840556059710826),
 '160308057': ('588785514', '1723593043', 0.0022489015473898129),
 '5680875': ('42500039', '42525919', 0.22387010939114038),
 '322783825': ('3295739694', '3295739688', 0.37827980635281871),
 '2216021172': ('42478180', '42478183', 0.011130940914012639),
 '2216021170': ('42478170', '42476753', 0.07847403744907494),
 '2216021171': ('42476753', '42478180', 0.06838206297326406),
 '337627139': ('1241183817', '3446961181', 0.044988058575439867),
 '46116280

In [154]:
#Write out weights to sql
#cur.execute("""
#CREATE TABLE bk_weights_crime(
#    edgeid BIGINT NOT NULL,
#    node1 BIGINT NOT NULL,
#    node2 BIGINT NOT NULL,
#    weight FLOAT(12,8) NOT NULL,
#    PRIMARY KEY (edgeid)
#    )
#    """)

add_edge= ("INSERT INTO bk_weights_crime "
           " (edgeid, node1, node2, weight)"
           " VALUES (%s, %s,%s, %s)")


In [155]:
for ed in crime_weights:
    edge_data = ed, crime_weights[ed][0], crime_weights[ed][1], crime_weights[ed][2]
    cur.execute(add_edge, edge_data)


db_connect.commit()
cur.close()
db_connect.close()